In [3]:
import pytesseract
from PIL import Image
import cv2
import numpy as np
import json
#import cohere
#import spacy
#import io
#import sys
#from pymongo import MongoClient  



#initially checking the accuracy of raw pytesseract for my use and then based on that we will be using pillow and open cv
#update: will have to use some preprocessing since colours can effect the reading

Installations:
1) Tesseract for Windows - Github -> Tessearct for Windows by UB Manheim
2) pip install pytesseract usin cmd as admin --> tesseract's wrapper for python
3) we will be using pytesseracts OCR -> optical character recogonition for the text extraction
4) Installed cohere API -pip install cohere -> not using now
5) Installed OpenAI API -pip install openai -> not using now
6) Installed spacy -pip install spacy -> open source NLP library -> not using now
7) installed Google Gen AI package -pip install -q -U google-generativeai
8) installed MongoDB for python -pip isntall pymongo

references 
1) Chat GPT
2) https://pypi.org/project/pytesseract/ -> documentation of pytesseract



    SOME CELLS WILL BE COMMENTED OUT SINCE IT DID ITS WORK, AND I DONT WANT TO REPEAT AND ADD THINGS TO THE DB

In [4]:
#with some image preprocessing

# Configure the path to the Tesseract executable (Windows-specific)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# loading the image using open cv
image_path = 'Basic OCR Tests/testocr2.jpg'
image = cv2.imread(image_path)

# Convert the image to grayscale (improves OCR accuracy)
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Optional: Apply thresholding to improve text visibility
#_, threshold_image = cv2.threshold(gray_image, 150, 255, cv2.THRESH_BINARY)

# Convert to PIL Image format (for pytesseract)
pil_image = Image.fromarray(gray_image)

# running OCR to extract text
extracted_text = pytesseract.image_to_string(pil_image)


print("Extracted Text:\n", extracted_text)


Extracted Text:
 Cedric himself knew nothing
whatever about it. It had never been
even mentioned to him. He knew that
his papa had been an Englishman,
because his mamma had told him so;
but then his papa had died when he
was so little a boy that he could not
remember very much about him,
except that he was big, and had blue
eyes and a long mustache, and that it
was a splendid thing to be carried
around the room on his shoulder.




Accuracy mainly depends on how clear the image is
this is just me tyring to implement this idea, i have very minimal knowledge of opencv, pillow and tesseract
I am just using the specific functionalitiles of these libraries that I need

my plan is to make this thing read some random text and then save it into a database

future updates: 
1) will be to study image processing and then make the reading accuracy better,
2) make a webpage where used can upload the image and the read text will go into his database. (mongo db since is nosql and its the only nosql language i know and diff cards will have diff details so using sql there will be many null values or errors)
3) try and make that webpage into an app (find collaborators since idk shit abt web dev)


In [5]:
# going to go all out and no do any preprocessing on the images, just RAW image lets see what happens
#why im removing all image preprocessing is that, i dont know half the preprocessing that is happening and -
#- blatanntly copy paste code from the internet 
#removed converting to PIL image formate since the ocr function has gotten better over the years



# Configure the path to the Tesseract executable (Windows-specific)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# loading the image using open cv
image_path = 'Basic OCR Tests/testocr.png'
image = cv2.imread(image_path)

# Convert the image to grayscale (improves OCR accuracy)
#gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Optional: Apply thresholding to improve text visibility
#_, threshold_image = cv2.threshold(gray_image, 150, 255, cv2.THRESH_BINARY)

# Convert to PIL Image format (for pytesseract)
#pil_image = Image.fromarray(image)

# running OCR to extract text
extracted_text = pytesseract.image_to_string(image)


print("Extracted Text:\n", extracted_text)


Extracted Text:
 This is a lot of 12 point text to test the
ocr code and see if it works on all types
of file format.

The quick brown dog jumped over the
lazy fox. The quick brown dog jumped
over the lazy fox. The quick brown dog
jumped over the lazy fox. The quick
brown dog jumped over the lazy fox.



since the purpose of this project is to read business cards, we will need to test out some business cards and try to read data
1) just randomly read whatever is read as one whole string
2) use regex (idk regex, just used once in js lecture) and then try and split it into name, phno, email etc
3) if card images are not being read right then apply preprocessing

In [6]:
#since not preprocessing im gonna try and completely eliminate using open cv and read the image using pytesseract
image=Image.open("Basic OCR Tests/testocr.png")
extracted = pytesseract.image_to_string('Basic OCR Tests/testocr.png')
print(extracted)
#can traslate to diff languages-> refer documentation

This is a lot of 12 point text to test the
ocr code and see if it works on all types
of file format.

The quick brown dog jumped over the
lazy fox. The quick brown dog jumped
over the lazy fox. The quick brown dog
jumped over the lazy fox. The quick
brown dog jumped over the lazy fox.



Well since that worked we can completely eliminate open cv and pillow since i dont wanna use them.

Now to test some business cards since that is our end-goal
issues being business card will have different colours, so we will have to use image preporcessing 
lets upload a random business card image and check 

In [7]:
#testing a business card with colour near text to see if we need to do any image preprocessing
image=Image.open("business cards/bs_card_1.jpg")
extracted=pytesseract.image_to_string("business cards/bs_card_1.jpg")
print(extracted)

prachi@mail.com

PR GH

Lower Parel, Mumbai 400033




her mail is right, but her name is wrong, and address is right, which proves the colour issue

In [8]:
#testing a business card with colour near text to see if we need to do any image preprocessing
image=Image.open("business cards/bs_card_2.png")
extracted_businesscard=pytesseract.image_to_string("business cards/bs_card_2.png")
print(extracted_businesscard)

JAMES SMITH

SALES MANAGER

& 123 123 123

® jamessmith@spc.com

pc.com




the phone number logo was mistook for an "&" symbol and a dot in the mail is missed
ignore thr last "pc.com" it was covered in the test image itself

In [9]:
#going to try with a business card I found lying around, it has more text and more vibrant colours
image=Image.open("business cards/bs_card_3.jpg")
extracted=pytesseract.image_to_string("business cards/bs_card_3.jpg")

print(extracted)

©
K K OPTICS

explore the world of vision

multi branded Lenses
& quality frames

Jetty Road, Vallikavu, Karunagapally - 690 525



ok that worked well, only the mail wasnt captured, but even i couldnt see it with my eyes
1) so this image was taken close up, which means close up image has the best accuracy
2) will need image preprocessing, even after image was taken close up, it still had errors

In [10]:
#going to try out on my uni id card which has so many texts
image=Image.open("business cards/cld_id_card.jpg")
extracted=pytesseract.image_to_string("business cards/cld_id_card.jpg")

print(extracted)

@AMRITA

VISHWA VIDYAPEETHAM

Amritapuri Campus

Arun Anilkumar Manjula
BCADS

School of Computing
AM.EN.U3CDS22020

2022-2025

| 00008




In [11]:
#going to try out on my uni id card which has so many texts
image=Image.open("business cards/barbeque_company_card.jpg")
extracted=pytesseract.image_to_string("business cards/barbeque_company_card.jpg")

print(extracted)

BARBE COMPANY

BS EVENTS ESTD. 2012
SMOKE @ GRILL
NALLILA
MOB ; 7012352100, 9074212771 -




In [12]:
#going to try out on my uni id card which has so many texts
image=Image.open("business cards/barbeque_company_card.jpg")
extracted=pytesseract.image_to_string("business cards/barbeque_company_card.jpg")

print(extracted)

BARBE COMPANY

BS EVENTS ESTD. 2012
SMOKE @ GRILL
NALLILA
MOB ; 7012352100, 9074212771 -




findings:
1) quality of pic and how close up it is
2) how clear the text is on the card

CONCLUSION WITHOUT IMAGE  PREPROCESSING : THE CLOSER THE IMAGES ARE CAPTURED THE BETTER THE READING ACCURACY (accuracy is still minimal)

HOW GOOD THE TEXT IS ON THE IMAGE (SIZE OF TEXT)


update: since its of no use storing the extracted texts directly to database we will be exploring other ways to extract what is what from texts:
1) import a NLP model like BERT 
2) Use openAI api to send the extarcted text to openAI and the AI will seperate it into a JSON format using which we can directly store it to mongoDB --> refer openAI API documentation and the official API page
3) Update: cant afford OpenAI API so we will be using a free verstion of Cohere API

In [13]:


# # Initialize Cohere client
# co = cohere.Client('api_key')  #our cohere free api key #wont be on github

# # Predefined examples for classification
# examples = [
#     #company names
#       {"text": "TATA CONSULTANCY SERVICES", "label": "Company Name"},
#     {"text": "INFOSYS LIMITED", "label": "Company Name"},
#     {"text": "WIPRO TECHNOLOGIES", "label": "Company Name"},
#     {"text": "RELIANCE INDUSTRIES", "label": "Company Name"},
#     {"text": "HDFC BANK", "label": "Company Name"},
#     {"text": "ICICI PRUDENTIAL", "label": "Company Name"},
#     {"text": "ADITYA BIRLA GROUP", "label": "Company Name"},
#     {"text": "AMUL", "label": "Company Name"},
#     {"text": "FLIPKART PRIVATE LIMITED", "label": "Company Name"},
#     {"text": "BYJU'S", "label": "Company Name"},
#     {"text": "BHARAT PETROLEUM", "label": "Company Name"},
#     {"text": "STATE BANK OF INDIA", "label": "Company Name"},
#     #person names
#     {"text": "ARUN KUMAR", "label": "Name"},
#     {"text": "PRIYA SHARMA", "label": "Name"},
#     #job titles
#     {"text": "SOFTWARE ENGINEER", "label": "Job Title"},
#     {"text": "MARKETING EXECUTIVE", "label": "Job Title"},
#     {"text": "Software Engineer", "label": "Job Title"},
#     {"text": "Project Manager", "label": "Job Title"},
#     {"text": "Data Scientist", "label": "Job Title"},
#     {"text": "AI Specialist", "label": "Job Title"},
#     {"text": "Business Analyst", "label": "Job Title"},
#     {"text": "Graphic Designer", "label": "Job Title"},
#     {"text": "Chartered Accountant", "label": "Job Title"},
#     {"text": "Operations Manager", "label": "Job Title"},
#     {"text": "Human Resources Manager", "label": "Job Title"},
#     {"text": "Product Manager", "label": "Job Title"},
#     {"text": "Digital Marketing Specialist", "label": "Job Title"},
#     {"text": "Content Writer", "label": "Job Title"},
#     {"text": "Technical Lead", "label": "Job Title"},
#     {"text": "Civil Engineer", "label": "Job Title"},
#     {"text": "Mechanical Engineer", "label": "Job Title"},
#     {"text": "Electrical Engineer", "label": "Job Title"},
#     {"text": "Marketing Executive", "label": "Job Title"},
#     {"text": "Chief Executive Officer", "label": "Job Title"},
#     {"text": "Managing Director", "label": "Job Title"},
#     {"text": "Sales Executive", "label": "Job Title"},
#     {"text": "Legal Advisor", "label": "Job Title"},
#     {"text": "Research Scientist", "label": "Job Title"},
#     {"text": "IT Support Specialist", "label": "Job Title"},
#     {"text": "Professor", "label": "Job Title"},
#     #phone numbers
#     {"text": "+91 9876543210", "label": "Phone Number"},
#     {"text": "+91 9123456789", "label": "Phone Number"},
#     #email
#     {"text": "arun.kumar@example.in", "label": "Email"},
#     {"text": "priya.sharma@company.co.in", "label": "Email"},\
#     #address
#     {"text": "Jetty Road, Vallikavu, Karunagapally - 690525", "label": "Address"},
#     {"text": "MG Road, Bengaluru, Karnataka - 560001", "label": "Address"},
#     #idk
#     {"text": "www.example.in", "label": "Website"},
#     {"text": "www.company.co.in", "label": "Website"}
# ]



# # Text extracted from a business card  (will add it to a function later on)
# text = """
# K K OPTICS
# explore the world of vision
# multi branded Lenses
# & quality frames
# Jetty Road, Vallikavu, Karunagapally - 690 525
# """

# # Split the input text into lines  -> to avoid spaces
# lines = text.strip().split("\n")

# # dientifying from our texts
# response = co.classify(inputs=lines, examples=examples)

# # Structure the output into a JSON-like dictionary
# classified_data = {}
# for line, classification in zip(lines, response.classifications):
#     classified_data[classification.prediction] = line

# # Display the structured data
# print("Classified Data (JSON):")
# print(json.dumps(classified_data, indent=4))


UPDATE: issues with cohere..it isnt a generative AI, which means it will classify based on examples only, which is difficult
1) Varying variety of names in india
2) if we need to make example based prediction accurate it will require soo much examples and it wont be worth it
3) need GEN AI apis like gpt or claude

update: while searching the pricing for OPEN AI GPT I found this github page
- https://github.com/ayaka14732/ChatGPTAPIFree  - Goal to make AI accessible for all but it had stopped due to funding issues
- https://github.com/ztjhz/BetterChatGPT - they built a new project on top of that this is a free chat GPT AI, i think u have to run it locally

update: running it locally still requires thr paid API keys from open AI
I am planning to use ML models- pre trained ofc, specifically Named Entity Recogonition Models (NER) for this

In [14]:
# # we sill be testing out spacy NER without any training i.e in its default form
# # python -m spacy download en_core_web_sm  # Download a pre-trained English model



# # Load SpaCy's pre-trained English model
# nlp = spacy.load("en_core_web_sm")

# # Input text
# text = """
# ©
# K K OPTICS

# explore the world of vision

# multi branded Lenses
# & quality frames

# Jetty Road, Vallikavu, Karunagapally - 690 525



# """

# # Process the text
# doc = nlp(text)

# # Extract named entities
# print("Named Entities, their Labels, and Positions:")
# for ent in doc.ents:
#     print(f"{ent.text} ({ent.label_}) [Start: {ent.start}, End: {ent.end}]")


update: this is trained only for general use and not for specific extratction
- I will be going back to generative AI API's

update: google cloud API's are there for free trial $300 credit for 90 days, just register with 2 rupees.
- got 25,336 rupee worth of credits  -> december 7th to march 9th
- planning to use Gemini API
- https://ai.google.dev/api?lang=python - Gemini API documentation for python

In [15]:
import google.generativeai as genai

genai.configure(api_key="api_key") #private key, I wont be putting it on github
model = genai.GenerativeModel("gemini-1.5-flash")
response = model.generate_content("Explain how AI works")
print(response.text)

c:\Users\aruna\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Artificial intelligence (AI) is a broad field encompassing many techniques, but at its core, AI aims to create systems that can perform tasks that typically require human intelligence.  These tasks include things like learning, problem-solving, decision-making, perception, and natural language understanding.  There isn't one single "how AI works," but rather a collection of approaches:

**1. Machine Learning (ML):** This is the most prominent approach currently. Instead of being explicitly programmed, ML systems learn from data.  They identify patterns and relationships within the data to make predictions or decisions.  There are several types of ML:

* **Supervised Learning:** The algorithm is trained on a labeled dataset, meaning each data point is tagged with the correct answer.  The algorithm learns to map inputs to outputs based on these examples.  Examples include image classification (labeling images as "cat" or "dog") and spam detection.
* **Unsupervised Learning:** The algorit

worked, now to make it take the text and classify it

In [16]:
#testing 1 with business card --> barbeque company
import google.generativeai as genai

# Configure API key
genai.configure(api_key="api_key") #private key, I wont be putting it on github

# Define the input text
input_text = """
BARBE COMPANY
BS EVENTS ESTD. 2012
SMOKE @ GRILL
NALLILA
MOB ; 7012352100, 9074212771 -
"""

# Define a prompt for classification
prompt = f"""
Classify the following text into the following categories:
1. Company Name
2. Tagline (if any)
3. Location
4. Phone Numbers
5. Miscellaneous Information.

Input:
{input_text}

Output:
"""

# Generate the response
model = genai.GenerativeModel("gemini-1.5-flash")
response = model.generate_content(prompt)

# Print the classified output
print("Classified Output:")
print(response.text)


Classified Output:
Here's a classification of the provided text:

1. **Company Name:** BARBE COMPANY, BS EVENTS

2. **Tagline (if any):** SMOKE @ GRILL

3. **Location:** NALLILA

4. **Phone Numbers:** 7012352100, 9074212771

5. **Miscellaneous Information:** ESTD. 2012 (Establishment year)



Trying to get the output in a json format for direct upload to mongodb

In [17]:
#testing 2 with business card -> kk optics
import google.generativeai as genai

# Configure API key
genai.configure(api_key="api_key") #private key, I wont be putting it on github

# Define the input text
input_text = """
©
K K OPTICS

explore the world of vision

multi branded Lenses
& quality frames

Jetty Road, Vallikavu, Karunagapally - 690 525
"""

# Define a prompt for classification
prompt = prompt =  f"""
Classify the following text into the following categories depending on if you feel like that value is present or not:
1. Company Name
2. Person Name
3. Job Title
4. Email
5. Location
6. Phone Numbers
7. Website
and more based on what you feel the info is about.

and return it in a format for storing it in a MongoDb database

Input:
    {input_text}
"""

# Generate the response
model = genai.GenerativeModel("gemini-1.5-flash")
response = model.generate_content(prompt)

# Print the classified output
print("Classified Output:")
print(response.text)


Classified Output:
```json
{
  "companyName": "K K OPTICS",
  "companyDescription": "multi branded Lenses & quality frames",
  "address": {
    "street": "Jetty Road",
    "locality": "Vallikavu",
    "city": "Karunagapally",
    "postalCode": "690 525"
  },
  "website": null, 
  "email": null,
  "phoneNumbers": [],
  "personNames": [],
  "jobTitles": []
}
```



works well! -> and i havent been charged yet
- we will be doing a few more runs tro fine tune our AI response
- need to tell it to give us our output in a json format

The following code is a test to upload our data to a mongo db server that is hosted locally using MongoDB compass
reference - https://www.mongodb.com/docs/languages/python/pymongo-driver/current/write-operations/#std-label-pymongo-write

In [18]:


# Connect to local MongoDB instance
client = MongoClient("mongodb://localhost:27017/") #our local mongodb server connection key

# Access a database
db = client["business_cards"]  # our database name

# Access a collection
collection = db["card_details"]  # oru collection name

# Insert a document
data ={
  "Company Name": "KK OPTICS",
  "Location": "Jetty Road, Vallikavu, Karunagapally - 690 525",
  "Description": "Multi branded Lenses & quality frames",
  "Tagline": "explore the world of vision" 
}
collection.insert_one(data)

print("Document inserted successfully!")


Document inserted successfully!


worked! the example document is inserted to our database

Next objective is to make both the OCR Reading and the Info Classification a function and then use the info of the OCR reading directly into the Classification function

Before we continue we will have to work on formatting our json texts since they are being returned as strings
I will be working that out here in order to not make the automation file conjested

The extract function taken from the functions oage

In [19]:
#function from functions page

#function for image extraction
def image_extract(image_location):
    pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe' #tesseract path on pc
    image=Image.open(image_location)
    extracted_businesscard=pytesseract.image_to_string(image)
    return extracted_businesscard
   # print(extracted_businesscard)
    

image_extract("business cards/microsoft_business_card.png")
#worked

'Jordan\n\nMitchell\n\nCEO\n\nVanArsdel, Ltd.\n\n5678 Main St. New York, NY 90210\n212-555-0199\nwww.vanarsdelltd.com\n\nQ@evdanavanarsdetta.com\n'

Function for classifying taken from function page

In [20]:

#FUNCTION MADE IN FUNCTION PAGE

#testing with new promts
def classify_text(image_location):
    input_text = image_extract(image_location)  # testing 2 with business card -> kk optics

    # Configure API key
    genai.configure(api_key="api_key")  # private key, I won't be putting it on github

    # Define a prompt for classification
    prompt =  f"""
Classify the following text into the following categories depending on if you feel like that value is present or not:
1. Company Name
2. Person Name
3. Job Title
4. Email
5. Location
6. Phone Numbers
7. Website
and more based on what you feel the info is about.
and return it in a format for storing it in a MongoDb database
if there are null values give it 0 instead of null
Input:
    {input_text}

"""


    # Generate the response
    model = genai.GenerativeModel("gemini-1.5-flash")
    response = model.generate_content(prompt)
   # print(response.text)
    return response.text





In [21]:
nini=classify_text("business cards/bs_card_3.jpg")


In [22]:
print(nini)

```json
{
  "Company Name": "KK OPTICS",
  "Person Name": 0,
  "Job Title": 0,
  "Email": 0,
  "Location": "Jetty Road, Vallikavu, Karunagapally - 690 525",
  "Phone Numbers": 0,
  "Website": 0,
  "Products": "multi branded Lenses & quality frames",
  "Description": "explore the world of vision"
}
```



In [23]:
# extract_json_content is a funtion to parse our text as a dict since the AI returns it in str and str cant be uploaded into MongoDB
nini=extract_json_content(nini)

In [24]:
print(nini)

{'Company Name': 'KK OPTICS', 'Person Name': 0, 'Job Title': 0, 'Email': 0, 'Location': 'Jetty Road, Vallikavu, Karunagapally - 690 525', 'Phone Numbers': 0, 'Website': 0, 'Products': 'multi branded Lenses & quality frames', 'Description': 'explore the world of vision'}


In [25]:
type(nini)

dict

Will be commenting the below code (adding a text to the database)
- it did it's work of taking a variable 

In [26]:
# #trying to put that into the database

# # Connect to local MongoDB instance
# client = MongoClient("mongodb://localhost:27017/") #our local mongodb server connection key

# # Access a database
# db = client["business_cards"]  # our database name

# # Access a collection
# collection = db["card_details"]  # oru collection name

# # Insert a document
# data =nini
# collection.insert_one(data)

# print("Document inserted successfully!")



Trying to add the made by our function to the database
REASON: updated the promt to return 0 when null due to MongoDB not reading the null values

In [27]:

# # Connect to local MongoDB instance
# client = MongoClient("mongodb://localhost:27017/") #our local mongodb server connection key

# # Access a database
# db = client["business_cards"]  # our database name

# # Access a collection
# collection = db["card_details"]  # oru collection name

# # Insert a document
# data ={
#   "Company Name": "KK OPTICS",
#   "Person Name": 0,
#   "Job Title": 0,
#   "Email": 0,
#   "Location": "Jetty Road, Vallikavu, Karunagapally - 690 525",
#   "Phone Numbers": 0,
#   "Website": 0,
#   "Industry": "Optics",
#   "Products": "Multi branded Lenses & quality frames" 
# }
# collection.insert_one(data)

# print("Document inserted successfully!")


Made the extract_json_content function to parse the AI given text (str) to a (dict) for upload into MongoDB
- This was supposed to be done before making the database add function but we faced the parsing issue after that

In [28]:
#trying to remove the ''' json '''' in the generated text


def extract_json_content(raw_text):
    try:
        # Find the first occurrence of '{' and the last occurrence of '}'
        start_index = raw_text.find('{')
        end_index = raw_text.rfind('}')
        
        # Extract the JSON content
        json_content = raw_text[start_index:end_index + 1]
        
        # Parse to ensure it's valid JSON
        parsed_json = json.loads(json_content)
        
        # Return the parsed JSON
        return parsed_json
    except Exception as e:
        print(f"Error parsing JSON: {e}")
        return None
    


# Example usage
raw_text = """
'''json
{
  "Company Name": "KK OPTICS",
  "Person Name": 0,
  "Job Title": 0,
  "Email": 0,
  "Location": "Jetty Road, Vallikavu, Karunagapally - 690 525",
  "Phone Numbers": 0,
  "Website": 0,
  "Industry": "Optics",
  "Products": "Multi branded Lenses & quality frames"
}
'''
"""

extract_json_content(raw_text)






{'Company Name': 'KK OPTICS',
 'Person Name': 0,
 'Job Title': 0,
 'Email': 0,
 'Location': 'Jetty Road, Vallikavu, Karunagapally - 690 525',
 'Phone Numbers': 0,
 'Website': 0,
 'Industry': 'Optics',
 'Products': 'Multi branded Lenses & quality frames'}

trying out io.StringIO from io module to capture the value of the print output since the return function is giving it in the wrong format
- update:did not work out, therefore removed the code.
- we instead gave the return statement to a variable and then the structure(of output text) did not change, which was our initial problem

Testing the json parsing function 

In [29]:
extract_json_content(raw_text)
#this can be uploaded to mongo db with our usual upload code, checked

{'Company Name': 'KK OPTICS',
 'Person Name': 0,
 'Job Title': 0,
 'Email': 0,
 'Location': 'Jetty Road, Vallikavu, Karunagapally - 690 525',
 'Phone Numbers': 0,
 'Website': 0,
 'Industry': 'Optics',
 'Products': 'Multi branded Lenses & quality frames'}

Thats it for the experiments, In the file:
1) We chose our OCR
2) We chose how to classify text (NLP or GEN AI)
3) We made a functiont to parse the text as a json file
4) We chose database (MongoDB) to store our data since diff business cards might have diff info we needed a NoSQL database and I am familiar with MongoDB

- The "functions.ipynb", is wherer all the functions were made and they were tested out here.
- The final output is provided in the file itself, pls do refer it